# [Task 2A: Multimodal Propagandistic Memes Classification](https://araieval.gitlab.io/task2/) at [ArabicNLP 2024](https://arabicnlp2024.sigarab.org/) @ACL 2024

@Author: Md. Arid Hasan

Given a text extracted from meme, categorize whether it is propagandistic or not.



### Downloading dataset from github

In [1]:
!wget https://gitlab.com/araieval/araieval_arabicnlp24/-/raw/main/task2/data/arabic_memes_propaganda_araieval_24_train.json
!wget https://gitlab.com/araieval/araieval_arabicnlp24/-/raw/main/task2/data/arabic_memes_propaganda_araieval_24_dev.json

--2024-03-22 05:40:32--  https://gitlab.com/araieval/araieval_arabicnlp24/-/raw/main/task2/data/arabic_memes_propaganda_araieval_24_train.json
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027033 (1003K) [text/plain]
Saving to: ‘arabic_memes_propaganda_araieval_24_train.json’

arabic_memes_propag 100%[===================>]   1003K  --.-KB/s    in 0.05s   

2024-03-22 05:40:32 (18.0 MB/s) - ‘arabic_memes_propaganda_araieval_24_train.json’ saved [1027033/1027033]

--2024-03-22 05:40:32--  https://gitlab.com/araieval/araieval_arabicnlp24/-/raw/main/task2/data/arabic_memes_propaganda_araieval_24_dev.json
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 150732 (147

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

#### importing required libraries and setting up logger

In [3]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [4]:
train_file = 'arabic_memes_propaganda_araieval_24_train.json'
validation_file = 'arabic_memes_propaganda_araieval_24_dev.json'
# test_file = 'arabic_memes_propaganda_araieval_24_test.json'

### Setting up the training parameters

In [5]:
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,
    remove_unused_columns=False,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no"
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16

In [6]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False

#### Defining the Model

In [7]:
model_name = 'distilbert-base-multilingual-cased'

#### setting the random seed

In [8]:
set_seed(training_args.seed)

#### Loading data files

In [9]:
import json
def read_data(fpath, is_test=False):
  if is_test:
    data = {'id': [], 'text': []}
    js_obj = json.load(open(fpath, encoding='utf-8'))
    for obj in js_obj:
      data['id'].append(obj['id'])
      data['text'].append(obj['text'])
  else:
    data = {'id': [], 'text': [], 'label': []}
    js_obj = json.load(open(fpath, encoding='utf-8'))
    for obj in js_obj:
      data['id'].append(obj['id'])
      data['text'].append(obj['text'])
      data['label'].append(obj['class_label'])
  return pd.DataFrame.from_dict(data)


l2id = {'not_propaganda': 0, 'propaganda': 1}
train_df = read_data(train_file)
train_df['label'] = train_df['label'].map(l2id)
train_df = Dataset.from_pandas(train_df)
validation_df = read_data(validation_file)
validation_df['label'] = validation_df['label'].map(l2id)
validation_df = Dataset.from_pandas(validation_df)
# test_df = read_data(test_file)
# #test_df['label'] = test_df['label'].map(l2id)
# test_df = Dataset.from_pandas(test_df)



#data_files = {"train": train_df, "validation": validation_df, "test": validation_df}
data_files = {"train": train_df, "validation": validation_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df} # , "test": test_df
)

INFO:__main__:loading a local file for train
INFO:__main__:loading a local file for validation


##### Extracting number of unique labels

In [10]:
# Labels
label_list = raw_datasets["train"].unique("label")
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

### Loading Pretrained Configuration, Tokenizer and Model

In [11]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

[INFO|configuration_utils.py:728] 2024-03-22 05:45:13,346 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/config.json
[INFO|configuration_utils.py:791] 2024-03-22 05:45:13,363 >> Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.38.2",
  "vocab_size": 119547
}



tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

[INFO|configuration_utils.py:728] 2024-03-22 05:45:13,498 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/config.json
[INFO|configuration_utils.py:791] 2024-03-22 05:45:13,503 >> Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.38.2",
  "vocab_size": 119547
}



vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2046] 2024-03-22 05:45:14,570 >> loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/vocab.txt
[INFO|tokenization_utils_base.py:2046] 2024-03-22 05:45:14,572 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/tokenizer.json
[INFO|tokenization_utils_base.py:2046] 2024-03-22 05:45:14,573 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-22 05:45:14,577 >> loading file special_tokens_map.json from cache at None
[INFO|tokenization_utils_base.py:2046] 2024-03-22 05:45:14,582 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/tokenizer_config.json
[INFO|configuration_

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

[INFO|modeling_utils.py:3257] 2024-03-22 05:45:22,028 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/9e90d6dd84b6a2e4d65e4d751baed6cd56578fd3/model.safetensors
[INFO|modeling_utils.py:3982] 2024-03-22 05:45:23,524 >> Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing

#### Preprocessing the raw_datasets

In [12]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/2143 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/312 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [13]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [14]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2143
})

#### Finalize the development/evaluation data for evaluating the model

In [15]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [ ]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [16]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

INFO:__main__:Sample 456 of the training set: {'id': 'data/arabic_memes_fb_insta_pinterest/Facebook/images/sanat.meme_/79517291_1444731972385670_3984977326792453161_n.jpg', 'text': 'في رسائل من طلاب جامعة عدن يشتكوا من سوء المناهج وتكديسها ورداءة الإنترنت ومفيش كهرباء عشان يتعلموا اون لاين مثل ماقلتوا..\nالخضر لصور\nيقلك في صفحة بالفيس بوك نظمت مسابقة لايكات وفازت جامعة عدن\nالخضر لصور\nالله عليكم يا طلاب جامعة عدن الاعزاء لولاكم لما حققنا هذا الفوز الساحق الف مبروك لنا جميعا\nالخضر لصور\n', 'label': 1, 'input_ids': [101, 10210, 773, 90244, 10289, 108962, 11086, 23215, 781, 22955, 793, 17444, 39494, 10429, 10289, 40443, 12611, 59901, 20623, 23984, 13027, 791, 10502, 66562, 25563, 10742, 791, 21172, 108096, 77369, 791, 10700, 21970, 11626, 786, 43598, 58267, 781, 39764, 793, 10502, 35263, 14556, 12084, 10582, 13879, 11294, 15593, 12441, 23927, 40041, 10429, 119, 119, 59901, 107166, 787, 29360, 793, 23927, 12497, 10210, 33220, 10909, 42472, 25563, 40218, 12497, 85532, 10502, 788, 49506, 

#### Get the metric function `accuracy`

In [17]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [18]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


#### Data Collator

In [19]:
data_collator = default_data_collator

#### Initialize our Trainer

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset, # if you have development and test set, uncomment this line
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


#### Training our model

In [21]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[INFO|trainer.py:1812] 2024-03-22 05:45:27,015 >> ***** Running training *****
[INFO|trainer.py:1813] 2024-03-22 05:45:27,020 >>   Num examples = 2,143
[INFO|trainer.py:1814] 2024-03-22 05:45:27,025 >>   Num Epochs = 1
[INFO|trainer.py:1815] 2024-03-22 05:45:27,030 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:1818] 2024-03-22 05:45:27,035 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:1819] 2024-03-22 05:45:27,040 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1820] 2024-03-22 05:45:27,041 >>   Total optimization steps = 134
[INFO|trainer.py:1821] 2024-03-22 05:45:27,044 >>   Number of trainable parameters = 135,326,210


Step,Training Loss


[INFO|trainer.py:2067] 2024-03-22 05:45:56,754 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)




#### Saving the tokenizer too for easy upload

In [22]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:3067] 2024-03-22 05:45:56,801 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:473] 2024-03-22 05:45:56,810 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:2454] 2024-03-22 05:45:59,232 >> Model weights saved in ./distilBERT_m/model.safetensors
[INFO|tokenization_utils_base.py:2459] 2024-03-22 05:45:59,236 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2468] 2024-03-22 05:45:59,238 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                    =        1.0
  total_flos               =    66095GF
  train_loss               =     0.5408
  train_runtime            = 0:00:29.74
  train_samples            =       2143
  train_samples_per_second =     72.057
  train_steps_per_second   =      4.506


#### Evaluating our model on validation/development data

In [23]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

INFO:__main__:*** Evaluate ***
[INFO|trainer.py:3376] 2024-03-22 05:45:59,346 >> ***** Running Evaluation *****
[INFO|trainer.py:3378] 2024-03-22 05:45:59,348 >>   Num examples = 312
[INFO|trainer.py:3381] 2024-03-22 05:45:59,349 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.7917
  eval_loss               =     0.4856
  eval_runtime            = 0:00:01.25
  eval_samples            =        312
  eval_samples_per_second =    248.546
  eval_steps_per_second   =     15.932


### Predecting the test data

In [24]:
# if the test set is available, you don't need to run this cell
predict_dataset = eval_dataset

In [25]:
id2l = {0:'not_propaganda', 1:'propaganda'}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = os.path.join(training_args.output_dir, f"task2A_TeamName.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\trun_id\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")

INFO:__main__:*** Predict ***
[INFO|trainer.py:3376] 2024-03-22 05:46:00,639 >> ***** Running Prediction *****
[INFO|trainer.py:3378] 2024-03-22 05:46:00,643 >>   Num examples = 312
[INFO|trainer.py:3381] 2024-03-22 05:46:00,645 >>   Batch size = 16


INFO:__main__:***** Predict results *****


In [26]:
ids[0]

'data/arabic_memes_fb_insta_pinterest/Instagram/IMAGES/ex.officiall/2019-10-25_17-08-21_UTC.jpg'

#### Saving the model into card

In [27]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:450] 2024-03-22 05:46:01,957 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7916666865348816}]}
